In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import gc

gc.collect()

df = pd.read_csv('datasets/cleaned_qa_dataset.csv')  # Adjust the path and method according to your dataset format

print(df.head())

# Splitting dataset into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.2) 


In [ ]:

from transformers import AutoTokenizer

import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2-xl") 

# Function to tokenize the QA pairs
def tokenize_qa_pairs(df):
    return tokenizer(
        df['question'].tolist(),
        df['answer'].tolist(), 
        truncation=True, 
        padding='max_length', 
        max_length=128,  # Adjust based on model and memory constraints
        return_tensors='pt'
    )
tokenizer.pad_token = tokenizer.eos_token

train_encodings = tokenize_qa_pairs(train_df)
val_encodings = tokenize_qa_pairs(val_df)


In [ ]:
# import torch
# del tokenizer

# torch.cuda.empty_cache()

In [ ]:
from torch.utils.data import Dataset

class QADataset(Dataset):
    def __init__(self, encodings, answers):
        self.encodings = encodings
        self.answers = answers

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        # For generative QA, encode the answers and use them as labels
        answer_encoding = tokenizer(self.answers[idx], truncation=True, padding='max_length', max_length=128, return_tensors="pt")
        item['labels'] = answer_encoding['input_ids'][0]
        return item

    def __len__(self):
        return len(self.answers)

train_dataset = QADataset(train_encodings, train_df['answer'].tolist())
val_dataset = QADataset(val_encodings, val_df['answer'].tolist())


In [ ]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2-xl")

import torch




training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    warmup_steps=500,
    weight_decay=0.01,
    # deepspeed='dp_config.json',  # Path to your DeepSpeed config
    deepspeed=None,
    logging_dir='./logs',
    evaluation_strategy='epoch',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)


In [ ]:
trainer.train()

In [ ]:
model_save_path = './new-model'
tokenizer_save_path = './new-tokenizer'

# Save the model
model.save_pretrained(model_save_path)

# Save the tokenizer
tokenizer.save_pretrained(tokenizer_save_path)